# Timeseries classification with a Transformer model

**Author:** [Theodoros Ntakouris](https://github.com/ntakouris)<br>
**Date created:** 2021/06/25<br>
**Last modified:** 2021/08/05<br>
**Description:** This notebook demonstrates how to do timeseries classification using a Transformer model.

## Introduction

This is the Transformer architecture from
[Attention Is All You Need](https://arxiv.org/abs/1706.03762),
applied to timeseries instead of natural language.

This example requires TensorFlow 2.4 or higher.

## Load the dataset

We are going to use the same dataset and preprocessing as the
[TimeSeries Classification from Scratch](https://keras.io/examples/timeseries/timeseries_classification_from_scratch)
example.

In [1]:
import numpy as np
import keras
from keras import layers


def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

## Build the model

Our model processes a tensor of shape `(batch size, sequence length, features)`,
where `sequence length` is the number of time steps and `features` is each input
timeseries.

You can replace your classification RNN layers with this one: the
inputs are fully compatible!

We include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.

The projection layers are implemented through `keras.layers.Conv1D`.

In [2]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


The main part of our model is now complete. We can stack multiple of those
`transformer_encoder` blocks and we can also proceed to add the final
Multi-Layer Perceptron classification head. Apart from a stack of `Dense`
layers, we need to reduce the output tensor of the `TransformerEncoder` part of
our model down to a vector of features for each data point in the current
batch. A common way to achieve this is to use a pooling layer. For
this example, a `GlobalAveragePooling1D` layer is sufficient.

In [3]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


## Train and evaluate

In [4]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

#callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 500, 1)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 500, 1)         │          7,169 │ input_layer[0][0],     │
│ (MultiHeadAttention)      │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 500, 1)         │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 500, 1)         │              2 │ dropout_1[0][0]        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 500, 1)         │              0 │ layer_normalization[0… │
│                           │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 500, 4)         │              8 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 500, 4)         │              0 │ conv1d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 500, 1)         │              5 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 500, 1)         │              2 │ conv1d_1[0][0]         │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 500, 1)         │              0 │ layer_normalization_1… │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 500, 1)         │          7,169 │ add_1[0][0],           │
│ (MultiHeadAttention)      │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 500, 1)         │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 500, 1)         │              2 │ dropout_4[0][0]        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 500, 1)         │              0 │ layer_normalization_2… │
│                           │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 500, 4)         │              8 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)  

 Total params: 29,258 (114.29 KB)

 Trainable params: 29,258 (114.29 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
!pip install codecarbon
from codecarbon import EmissionsTracker

In [7]:
#start emissions tracking
tracker = EmissionsTracker()
tracker.start()

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
)

#stop emissions tracking
emissions: float = tracker.stop()
print(f"Emissions during training: {emissions} kg CO2")

[codecarbon WARNING @ 05:11:52] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 05:11:52] [setup] RAM Tracking...
[codecarbon INFO @ 05:11:52] [setup] CPU Tracking...
[codecarbon WARNING @ 05:11:53] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 05:11:53] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 05:11:53] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 05:11:53] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 05:11:53] [setup] GPU Tracking...
[codecarbon INFO @ 05:11:53] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 05:11:53] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

Epoch 1/150


[codecarbon INFO @ 05:11:56] Energy consumed for RAM : 0.000167 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 05:11:56] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 05:11:56] Energy consumed for All CPU : 0.000708 kWh
[codecarbon INFO @ 05:11:56] Energy consumed for all GPUs : 0.000486 kWh. Total GPU Power : 29.658663380701313 W
[codecarbon INFO @ 05:11:56] 0.001361 kWh of electricity used since the beginning.
[codecarbon INFO @ 05:12:09] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 05:12:09] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 05:12:09] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 05:12:09] Energy consumed for all GPUs : 0.000128 kWh. Total GPU Power : 30.68358715838164 W
[codecarbon INFO @ 05:12:09] 0.000347 kWh of electricity used since the beginning.
[codecarbon INFO @ 05:12:11] Energy consumed for RAM : 0.000208 kWh. RAM Power : 

17/45 ━━━━━━━━━━━━━━━━━━━━ 8s 287ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.5320

[codecarbon INFO @ 05:12:24] Energy consumed for RAM : 0.000083 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 05:12:24] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 05:12:24] Energy consumed for All CPU : 0.000354 kWh
[codecarbon INFO @ 05:12:24] Energy consumed for all GPUs : 0.000345 kWh. Total GPU Power : 52.17263059735455 W
[codecarbon INFO @ 05:12:24] 0.000783 kWh of electricity used since the beginning.


...

38/45 ━━━━━━━━━━━━━━━━━━━━ 2s 310ms/step - loss: 0.6924 - sparse_categorical_accuracy: 0.5230

[codecarbon INFO @ 06:00:41] Energy consumed for RAM : 0.008287 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 06:00:41] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 06:00:41] Energy consumed for All CPU : 0.035222 kWh
[codecarbon INFO @ 06:00:41] Energy consumed for all GPUs : 0.052734 kWh. Total GPU Power : 62.2663603651698 W
[codecarbon INFO @ 06:00:41] 0.096243 kWh of electricity used since the beginning.


45/45 ━━━━━━━━━━━━━━━━━━━━ 19s 340ms/step - loss: 0.6924 - sparse_categorical_accuracy: 0.5216 - val_loss: 0.6930 - val_sparse_categorical_accuracy: 0.5090


[codecarbon INFO @ 06:00:45] Energy consumed for RAM : 0.008137 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 06:00:45] Delta energy consumed for CPU with constant : 0.000067 kWh, power : 42.5 W
[codecarbon INFO @ 06:00:45] Energy consumed for All CPU : 0.034584 kWh
[codecarbon INFO @ 06:00:45] Energy consumed for all GPUs : 0.052332 kWh. Total GPU Power : 70.19028784980856 W
[codecarbon INFO @ 06:00:45] 0.095053 kWh of electricity used since the beginning.


Emissions during training: 0.0447493108187012 kg CO2


In [8]:
model.evaluate(x_test, y_test, verbose=1)

42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - loss: 0.6919 - sparse_categorical_accuracy: 0.5299


[0.692647397518158, 0.5159090757369995]

In [9]:
emissions_data = tracker.final_emissions_data
print(f"Energy consumed: {emissions_data.energy_consumed:.6f} kWh")
print(f"Execution Time: {emissions_data.duration:.2f} seconds")
print(f"Emissions: {emissions_data.emissions} kg CO2")

Energy consumed: 0.095053 kWh
Execution Time: 2931.01 seconds
Emissions: 0.0447493108187012 kg CO2


## Conclusions

In about 110-120 epochs (25s each on Colab), the model reaches a training
accuracy of ~0.95, validation accuracy of ~84 and a testing
accuracy of ~85, without hyperparameter tuning. And that is for a model
with less than 100k parameters. Of course, parameter count and accuracy could be
improved by a hyperparameter search and a more sophisticated learning rate
schedule, or a different optimizer.